# Analysing CIA Factbook Data Using SQL

In this mini project, I will work with data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/) which is a compendium of statistics about all of countries on Earth. The Factbook contains demographics information like the following:
- `population` - the global population.
- `population_growth` - the annual population growth rate, as a percentage.
- `area` - the total land and water area.


First, I will use the following code to connect the Jupyter Notebook to the database `factbook.db` file.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

To see the tables in the database, we can run the query like this:

In [2]:
%%sql
SELECT *
FROM sqlite_master
WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


We have a table called `facts` in the database. Let's try to run a query to return the first five rows of the table:

In [3]:
%%sql
SELECT *
FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns: 
- `name` - the name of the country.
- `area` - the country's total area (both land and water).
- `area_land` - the country's land area in square kilometers.
- `area_water` - the country's waterarea in square kilometers.
- `population` - the country's population.
- `population_growth` - the country's population growth as a percentage.
- `birth_rate` - the country's birth rate, or the number of births per year per 1,000 people.
- `death_rate` - the country's death rate, or the number of  death per year per 1,000 people.

I will write a query that returns the follows:
- Minimum population
- Maximum population
- Minimum population growth
- Maximum population growth

In [4]:
%%sql
SELECT MIN(population) AS Min_P, MAX(population) AS Max_P, MIN(population_growth) AS Min_P_G, MAX(population_growth) AS Max_P_G
FROM facts

 * sqlite:///factbook.db
Done.


Min_P,Max_P,Min_P_G,Max_P_G
0,7256490011,0.0,4.02


A few interesting things from the summary statistics above:
- There is a country with a population of `0`
- There is a country with a population of `7256490011` (or more than 7.2 billion people)

I will use subqueries to zoom in on just these countries without using specific values.

In [5]:
%%sql
SELECT name
FROM facts
WHERE population = (
    SELECT MIN(population) 
    FROM facts
);

 * sqlite:///factbook.db
Done.


name
Antarctica


In [6]:
%%sql
SELECT name
FROM facts
WHERE population = (
    SELECT MAX(population)
    FROM facts
);

 * sqlite:///factbook.db
Done.


name
World


Seems like the table includes the whole world as a country. Let's exclude the whole world and recompute the summary staticstics including the following:
- Minimum population
- Maximum population
- Minimum population growth
- Maximum population growth

In [7]:
%%sql
SELECT MIN(population) AS MIN_P, MAX(population) AS MAX_P, MIN(population_growth) AS MIN_P_G, MAX(population_growth) AS MAX_P_G
FROM (
    SELECT *
    FROM facts
    WHERE name is not 'World'
)

 * sqlite:///factbook.db
Done.


MIN_P,MAX_P,MIN_P_G,MAX_P_G
0,1367485388,0.0,4.02


Cool, now let's calculate the average value for the following columns:
- `population`
- `area`

In [8]:
%%sql
SELECT AVG(population) AS AVG_P, AVG(area) AS AVG_Area
FROM (
    SELECT *
    FROM facts
    WHERE name is not 'World'
)

 * sqlite:///factbook.db
Done.


AVG_P,AVG_Area
32242666.56846473,555093.546184739


To finish, I will build on the query wrote above to find countries that are densely populated. I will identify countries that have the following:
- Above-average values for population.
- Below-average values for area.


In [9]:
%%sql
SELECT *
FROM facts
WHERE population > (
    SELECT AVG(population)
    FROM (
        SELECT *
        FROM facts
        WHERE name is not 'World'
    )
)
AND
area < (
    SELECT AVG(area)
    FROM (
        SELECT *
        FROM facts
        WHERE name is not 'World'
    )
)

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


Some of these countries are generally known to be densely populated, so we have confidence in our results!

Now I want to figure out which countries will add the most people to their populations next year. We need to calculate the number of people added based on the current population and the population growth. 

In [20]:
%%sql
SELECT name, CAST((population / 100) * population_growth AS INTEGER) AS Num_Added_People
FROM (
    SELECT *
    FROM facts
    WHERE name <> 'World'
)
ORDER BY Num_Added_People DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,Num_Added_People
India,15270685
China,6153683
Nigeria,4448269
Pakistan,2906652
Ethiopia,2874561
Bangladesh,2703323
United States,2506676
Indonesia,2355141
"Congo, Democratic Republic of the",1944689
Philippines,1626072


We can see the India will add the most people in their population next year.

This time I want to know about areas of countries. Which countries have the highest ratios of water to land? Which countries have more water than land? Let's figure it out!

In [23]:
%%sql
SELECT name, area, area_land, area_water, ROUND((CAST(area_water AS Float) * 100)/area, 2) AS water_ratio
FROM (
    SELECT *
    FROM facts
    WHERE name <> 'World'
)
ORDER BY water_ratio DESC
LIMIT 10;

 * sqlite:///factbook.db
Done.


name,area,area_land,area_water,water_ratio
British Indian Ocean Territory,54400,60,54340,99.89
Virgin Islands,1910,346,1564,81.88
Puerto Rico,13791,8870,4921,35.68
"Bahamas, The",13880,10010,3870,27.88
Guinea-Bissau,36125,28120,8005,22.16
Malawi,118484,94080,24404,20.6
Netherlands,41543,33893,7650,18.41
Uganda,241038,197100,43938,18.23
Eritrea,117600,101000,16600,14.12
Liberia,111369,96320,15049,13.51


In [24]:
%%sql
SELECT name, area, area_land, area_water
FROM (
    SELECT *
    FROM facts
    WHERE name <> 'World'
)
WHERE area_water > area_land;

 * sqlite:///factbook.db
Done.


name,area,area_land,area_water
British Indian Ocean Territory,54400,60,54340
Virgin Islands,1910,346,1564


Great! We have figured out the top 10 countries where have the highest ratios of water to land and the countries have more water than land.